In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agri-yield-predictor-1/data_loo_stdscaler
/kaggle/input/agri-yield-predictor-1/__results__.html
/kaggle/input/agri-yield-predictor-1/data_onehot_stdscaled
/kaggle/input/agri-yield-predictor-1/data_loo_minmax
/kaggle/input/agri-yield-predictor-1/__notebook__.ipynb
/kaggle/input/agri-yield-predictor-1/data_onehot_minmax
/kaggle/input/agri-yield-predictor-1/__output__.json
/kaggle/input/agri-yield-predictor-1/custom.css
/kaggle/input/agri-yield-predictor-1/__results___files/__results___20_0.png
/kaggle/input/agri-yield-predictor-1/__results___files/__results___19_0.png
/kaggle/input/agri-yield-predictor-1/__results___files/__results___22_0.png
/kaggle/input/agri-yield-predictor-1/__results___files/__results___21_0.png
/kaggle/input/agri-yield-predictor-1/__results___files/__results___17_0.png
/kaggle/input/agri-yield-predictor-1/__results___files/__results___18_0.png
/kaggle/input/crop-yield-and-environmental-factors-2014-2023/crop_yield_dataset.csv


Regression Evaluation Metrics — Explanation & Insights

1️.  **R² Score (Coefficient of Determination)**

Range: 0 ≤ R² ≤ 1 (can be negative if the model performs worse than the mean prediction).

Meaning:

Shows how much of the variation in the target variable (y) is explained by the model.

Higher R² → better model performance.

Insight:

R² close to 1 → model fits data well.

R² close to 0 → model doesn’t explain much variance.

Negative R² → model performs worse than a simple mean predictor.

2️. **Adjusted R²**

Range: usually slightly less than R²; can be negative if the model is poor.

Meaning:

Adjusts R² for the number of predictors (features).

Prevents misleadingly high R² when adding unnecessary variables.

Insight:

Use Adjusted R² when comparing models with different numbers of features.

If Adjusted R² decreases after adding a new feature → the new feature is not useful.

3️. **MAE (Mean Absolute Error)**

Range: 0 → ∞

Meaning:

Average of absolute differences between predicted and actual values.

Indicates how far off predictions are, on average.

Insight:

Easy to interpret — “on average, predictions are off by X units.”

Less sensitive to large outliers than MSE or RMSE.

4️. **MSE (Mean Squared Error)**

Range: 0 → ∞

Meaning:

Average of squared errors between actual and predicted values.

Penalizes larger errors more heavily due to squaring.

Insight:

Good for emphasizing big errors.

Difficult to interpret in original units (since it’s squared).

5️. **RMSE (Root Mean Squared Error)**

Range: 0 → ∞

Meaning:

Square root of MSE — gives error in same units as the target variable.

Insight:

Commonly used in regression reports.

Lower RMSE → better model accuracy.

Useful to compare with average crop yield — e.g., “RMSE of 3.5 means average prediction error ≈ 3.5 yield units.”

In [2]:
#Leave one out encoded min max scaled data

import pandas as pd
import os

# check what files exist
print(os.listdir("/kaggle/input/agri-yield-predictor-1"))

# load the file you need (add .csv extension if it has one)
data_loo_minmax = pd.read_csv("/kaggle/input/agri-yield-predictor-1/data_loo_minmax")

print("Loaded successfully")
print(data_loo_minmax.head())

['data_loo_stdscaler', '__results__.html', 'data_onehot_stdscaled', 'data_loo_minmax', '__notebook__.ipynb', '__results___files', 'data_onehot_minmax', '__output__.json', 'custom.css']
Loaded successfully
   Unnamed: 0        date  crop_type  soil_type  soil_pH  temperature  \
0           0  2014-01-01  40.966442  52.501041      0.4     0.402160   
1           1  2014-01-01  37.007957  33.402460      0.5     0.390129   
2           2  2014-01-01  36.923115  31.327259      0.0     0.244445   
3           3  2014-01-01  41.984588  33.408309      0.5     0.193090   
4           4  2014-01-01  32.445278  41.611338      0.3     0.332976   

   humidity  wind_speed         n         p         k  crop_yield  \
0  0.996494    0.495300  0.847826  0.833333  0.696970  104.871310   
1  1.000000    0.251013  0.108696  0.111111  0.090909   58.939796   
2  1.000000    0.458278  0.097826  0.250000  0.348485   32.970413   
3  1.000000    0.568686  0.217391  0.222222  0.272727   29.356115   
4  1.000000

In [3]:
data_loo_minmax

,Unnamed: 0,date,crop_type,soil_type,soil_pH,temperature,humidity,wind_speed,n,p,k,crop_yield,soil_quality
0,0,2014-01-01,40.966442,52.501041,0.4,0.402160,0.996494,0.495300,0.847826,0.833333,0.696970,104.871310,0.874403
1,1,2014-01-01,37.007957,33.402460,0.5,0.390129,1.000000,0.251013,0.108696,0.111111,0.090909,58.939796,0.355631
2,2,2014-01-01,36.923115,31.327259,0.0,0.244445,1.000000,0.458278,0.097826,0.250000,0.348485,32.970413,0.145392
3,3,2014-01-01,41.984588,33.408309,0.5,0.193090,1.000000,0.568686,0.217391,0.222222,0.272727,29.356115,0.427304
4,4,2014-01-01,32.445278,41.611338,0.3,0.332976,1.000000,0.479032,0.326087,0.250000,0.393939,22.221375,0.475085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25490,25490,2023-12-31,37.026160,41.613125,0.3,0.064709,1.000000,0.747856,0.326087,0.388889,0.393939,13.069169,0.498976
25491,25491,2023-12-31,41.967278,41.601361,0.3,0.381596,1.000000,0.291693,0.456522,0.527778,0.636364,73.323885,0.589761
25492,25492,2023-12-31,40.999772,31.326036,0.0,0.442798,0.928758,0.503711,0.576087,0.500000,0.454545,39.226521,0.239590
25493,25493,2023-12-31,37.421843,31.332520,0.0,0.043176,1.000000,0.593603,0.336957,0.250000,0.136364,6.067881,0.156314


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# --- Use your existing processed data ---
data = data_loo_minmax.copy()

# --- Convert 'date' column to numeric (if exists) ---
if 'date' in data.columns:
    data['date'] = pd.to_datetime(data['date'], errors='coerce')
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data = data.drop(columns=['date'])   # remove original date column (keep numeric parts)

# Define features (X) and target (y)
X = data.drop(columns=['crop_yield'])   # drop only the target column
y = data['crop_yield']                  # target variable

print("Features")
print(X.head())
print("  ")
print("Target")
print(y.head())

Features
   Unnamed: 0  crop_type  soil_type  soil_pH  temperature  humidity  \
0           0  40.966442  52.501041      0.4     0.402160  0.996494   
1           1  37.007957  33.402460      0.5     0.390129  1.000000   
2           2  36.923115  31.327259      0.0     0.244445  1.000000   
3           3  41.984588  33.408309      0.5     0.193090  1.000000   
4           4  32.445278  41.611338      0.3     0.332976  1.000000   

   wind_speed         n         p         k  soil_quality  year  month  day  
0    0.495300  0.847826  0.833333  0.696970      0.874403  2014      1    1  
1    0.251013  0.108696  0.111111  0.090909      0.355631  2014      1    1  
2    0.458278  0.097826  0.250000  0.348485      0.145392  2014      1    1  
3    0.568686  0.217391  0.222222  0.272727      0.427304  2014      1    1  
4    0.479032  0.326087  0.250000  0.393939      0.475085  2014      1    1  
  
Target
0    104.871310
1     58.939796
2     32.970413
3     29.356115
4     22.221375
Name: 

In [5]:
# --- Apply Polynomial Transformation ---
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# --- Split data ---
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# --- Train model ---
poly_model = LinearRegression()
poly_model.fit(X_train, y_train)

# --- Predict ---
y_pred = poly_model.predict(X_test)

# --- Evaluate ---
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Polynomial Regression Results for leave-one-out encode and min max scaled data:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")


Polynomial Regression Results for leave-one-out encode and min max scaled data:
R² Score: 0.9240
RMSE: 6.1973


In [6]:
# --- Optional: sample predictions ---
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print("\nSample Predictions:")
print(results.head())


Sample Predictions:
      Actual  Predicted
0  17.603608  20.079319
1  37.286219  49.342122
2  81.836093  78.063057
3  47.802599  46.269185
4  46.212115  51.700630


In [7]:
data_loo_minmax

,Unnamed: 0,date,crop_type,soil_type,soil_pH,temperature,humidity,wind_speed,n,p,k,crop_yield,soil_quality
0,0,2014-01-01,40.966442,52.501041,0.4,0.402160,0.996494,0.495300,0.847826,0.833333,0.696970,104.871310,0.874403
1,1,2014-01-01,37.007957,33.402460,0.5,0.390129,1.000000,0.251013,0.108696,0.111111,0.090909,58.939796,0.355631
2,2,2014-01-01,36.923115,31.327259,0.0,0.244445,1.000000,0.458278,0.097826,0.250000,0.348485,32.970413,0.145392
3,3,2014-01-01,41.984588,33.408309,0.5,0.193090,1.000000,0.568686,0.217391,0.222222,0.272727,29.356115,0.427304
4,4,2014-01-01,32.445278,41.611338,0.3,0.332976,1.000000,0.479032,0.326087,0.250000,0.393939,22.221375,0.475085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25490,25490,2023-12-31,37.026160,41.613125,0.3,0.064709,1.000000,0.747856,0.326087,0.388889,0.393939,13.069169,0.498976
25491,25491,2023-12-31,41.967278,41.601361,0.3,0.381596,1.000000,0.291693,0.456522,0.527778,0.636364,73.323885,0.589761
25492,25492,2023-12-31,40.999772,31.326036,0.0,0.442798,0.928758,0.503711,0.576087,0.500000,0.454545,39.226521,0.239590
25493,25493,2023-12-31,37.421843,31.332520,0.0,0.043176,1.000000,0.593603,0.336957,0.250000,0.136364,6.067881,0.156314


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet  # <-- this line fixes the error
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to print metrics
def print_metrics(name, y_true, y_pred, X):
    r2 = r2_score(y_true, y_pred)
    adj_r2 = 1 - (1 - r2) * (len(y_true) - 1) / (len(y_true) - X.shape[1] - 1)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"\n{name} Results:")
    print(f"R² Score: {r2:.4f}")
    print(f"Adjusted R²: {adj_r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")

# Lasso Regression
lasso = Lasso(alpha=0.005, max_iter=10000)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
print_metrics("Lasso Regression", y_test, y_pred_lasso, X_test)

# Ridge Regression
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
print_metrics("Ridge Regression", y_test, y_pred_ridge, X_test)

# ElasticNet Regression
elastic_net = ElasticNet(alpha=0.5, l1_ratio=0.5, max_iter=10000)
elastic_net.fit(X_train, y_train)
y_pred_elastic = elastic_net.predict(X_test)
print_metrics("ElasticNet Regression", y_test, y_pred_elastic, X_test)



Lasso Regression Results:
R² Score: 0.5747
Adjusted R²: 0.5735
MAE: 11.1795
MSE: 214.8755
RMSE: 14.6586

Ridge Regression Results:
R² Score: 0.5748
Adjusted R²: 0.5737
MAE: 11.1694
MSE: 214.7924
RMSE: 14.6558

ElasticNet Regression Results:
R² Score: 0.1731
Adjusted R²: 0.1709
MAE: 16.8695
MSE: 417.7299
RMSE: 20.4384


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("=== X_train ===")
print(X_train.head())

print("\n=== X_test ===")
print(X_test.head())

print("\n=== y_train ===")
print(y_train.head())

print("\n=== y_test ===")
print(y_test.head())


# --- Train Random Forest model ---
rf_model = RandomForestRegressor(
    n_estimators=200,     # number of trees
    random_state=42,
    max_depth=None,       # you can tune later
    min_samples_split=2,
    min_samples_leaf=1
)
rf_model.fit(X_train, y_train)

# --- Predict ---
y_pred = rf_model.predict(X_test)

# --- Evaluate ---
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# --- Print results ---
print("Random Forest Regression Results:")
print(f"R² Score: {r2:.4f}")
print(f"Adjusted R²: {adj_r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")

# --- Show few predictions ---
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print("\nSample Predictions:")
print(results.head())


=== X_train ===
       Unnamed: 0  crop_type  soil_type  soil_pH  temperature  humidity  \
13222       13222  40.993453  31.322879      0.0     0.604578  0.659099   
3295         3295  39.424055  33.404045      0.5     0.468944  0.885177   
4401         4401  39.434622  33.735036      1.0     0.187242  1.000000   
2310         2310  37.025151  31.330653      0.0     0.128926  1.000000   
15429       15429  37.107916  33.738559      1.0     0.044844  1.000000   

       wind_speed         n         p         k  soil_quality  year  month  \
13222    0.400144  0.576087  0.500000  0.454545      0.239590  2019      3   
3295     0.605187  0.108696  0.222222  0.181818      0.389078  2015      4   
4401     0.467248  0.434783  0.680556  0.681818      0.021160  2015     10   
2310     0.749204  0.217391  0.250000  0.242424      0.150853  2014     12   
15429    0.651051  0.576087  0.527778  0.409091      0.010239  2020      1   

       day  
13222    7  
3295     2  
4401     9  
2310     1  

In [19]:
data_loo_minmax

,Unnamed: 0,date,crop_type,soil_type,soil_pH,temperature,humidity,wind_speed,n,p,k,crop_yield,soil_quality
0,0,2014-01-01,40.966442,52.501041,0.4,0.402160,0.996494,0.495300,0.847826,0.833333,0.696970,104.871310,0.874403
1,1,2014-01-01,37.007957,33.402460,0.5,0.390129,1.000000,0.251013,0.108696,0.111111,0.090909,58.939796,0.355631
2,2,2014-01-01,36.923115,31.327259,0.0,0.244445,1.000000,0.458278,0.097826,0.250000,0.348485,32.970413,0.145392
3,3,2014-01-01,41.984588,33.408309,0.5,0.193090,1.000000,0.568686,0.217391,0.222222,0.272727,29.356115,0.427304
4,4,2014-01-01,32.445278,41.611338,0.3,0.332976,1.000000,0.479032,0.326087,0.250000,0.393939,22.221375,0.475085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25490,25490,2023-12-31,37.026160,41.613125,0.3,0.064709,1.000000,0.747856,0.326087,0.388889,0.393939,13.069169,0.498976
25491,25491,2023-12-31,41.967278,41.601361,0.3,0.381596,1.000000,0.291693,0.456522,0.527778,0.636364,73.323885,0.589761
25492,25492,2023-12-31,40.999772,31.326036,0.0,0.442798,0.928758,0.503711,0.576087,0.500000,0.454545,39.226521,0.239590
25493,25493,2023-12-31,37.421843,31.332520,0.0,0.043176,1.000000,0.593603,0.336957,0.250000,0.136364,6.067881,0.156314


In [9]:
#Leave one out encoded and standard scaled data

import pandas as pd
import os

# check what files exist
print(os.listdir("/kaggle/input/agri-yield-predictor-1"))

# load the file you need (add .csv extension if it has one)
data_loo_stdscaler = pd.read_csv("/kaggle/input/agri-yield-predictor-1/data_loo_stdscaler")

print("Loaded successfully")
print(data_loo_stdscaler.head())

['data_loo_stdscaler', '__results__.html', 'data_onehot_stdscaled', 'data_loo_minmax', '__notebook__.ipynb', '__results___files', 'data_onehot_minmax', '__output__.json', 'custom.css']
Loaded successfully
         date  crop_type  soil_type   soil_pH  temperature  humidity  \
0  2014-01-01  40.966442  52.501041 -0.122964    -0.260859  0.802313   
1  2014-01-01  37.007957  33.402460  0.182886    -0.315545  0.816279   
2  2014-01-01  36.923115  31.327259 -1.346363    -0.977727  0.816279   
3  2014-01-01  41.984588  33.408309  0.182886    -1.211154  0.816279   
4  2014-01-01  32.445278  41.611338 -0.428814    -0.575326  0.816279   

   wind_speed         n         p         k  crop_yield  soil_quality  
0   -0.469878  1.649420  1.473463  0.936043  104.871310      1.647104  
1   -2.437751 -1.468169 -1.475844 -1.409067   58.939796     -0.141463  
2   -0.768113 -1.514016 -0.908669 -0.412395   32.970413     -0.866303  
3    0.121288 -1.009700 -1.022104 -0.705534   29.356115      0.105642  
4 

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# --- Use your existing processed data ---
data1 = data_loo_stdscaler.copy()
print(data_loo_stdscaler.head())

# --- Convert 'date' column to numeric (if exists) ---
if 'date' in data1.columns:
    data1['date'] = pd.to_datetime(data1['date'], errors='coerce')
    data1['year'] = data1['date'].dt.year
    data1['month'] = data1['date'].dt.month
    data1['day'] = data1['date'].dt.day
    data1 = data1.drop(columns=['date'])   
    
# Define features (X) and target (y)
x1 = data1.drop(columns=['crop_yield'])   # drop only the target column
y1 = data1['crop_yield']                  # target variable

print("Features")
print(x1.head())
print("  ")
print("Target")
print(y1.head())

         date  crop_type  soil_type   soil_pH  temperature  humidity  \
0  2014-01-01  40.966442  52.501041 -0.122964    -0.260859  0.802313   
1  2014-01-01  37.007957  33.402460  0.182886    -0.315545  0.816279   
2  2014-01-01  36.923115  31.327259 -1.346363    -0.977727  0.816279   
3  2014-01-01  41.984588  33.408309  0.182886    -1.211154  0.816279   
4  2014-01-01  32.445278  41.611338 -0.428814    -0.575326  0.816279   

   wind_speed         n         p         k  crop_yield  soil_quality  
0   -0.469878  1.649420  1.473463  0.936043  104.871310      1.647104  
1   -2.437751 -1.468169 -1.475844 -1.409067   58.939796     -0.141463  
2   -0.768113 -1.514016 -0.908669 -0.412395   32.970413     -0.866303  
3    0.121288 -1.009700 -1.022104 -0.705534   29.356115      0.105642  
4   -0.600925 -0.551231 -0.908669 -0.236512   22.221375      0.270378  
Features
   crop_type  soil_type   soil_pH  temperature  humidity  wind_speed  \
0  40.966442  52.501041 -0.122964    -0.260859  0.8023

In [11]:

# --- Apply Polynomial Transformation ---
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(x1)

# --- Split data ---
X_train, X_test, y_train, y_test = train_test_split(X_poly, y1, test_size=0.2, random_state=42)

# --- Train model ---
poly_model = LinearRegression()
poly_model.fit(X_train, y_train)

# --- Predict ---
y_pred = poly_model.predict(X_test)

# --- Evaluate ---
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Polynomial Regression Results for leave-one-out encode and min max scaled data:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

# --- Optional: sample predictions ---
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print("\nSample Predictions:")
print(results.head())


Polynomial Regression Results for leave-one-out encode and min max scaled data:
R² Score: 0.9243
RMSE: 6.1853

Sample Predictions:
      Actual  Predicted
0  17.603608  19.863235
1  37.286219  48.806682
2  81.836093  77.141870
3  47.802599  47.672081
4  46.212115  52.492876


Model is underfitting R2 value is weak

In [16]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet  
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size=0.2, random_state=42)

# Function to print metrics
def print_metrics(name, y_true, y_pred, X):
    r2 = r2_score(y_true, y_pred)
    adj_r2 = 1 - (1 - r2) * (len(y_true) - 1) / (len(y_true) - X.shape[1] - 1)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"\n{name} Results:")
    print(f"R² Score: {r2:.4f}")
    print(f"Adjusted R²: {adj_r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")

# Lasso Regression
lasso = Lasso(alpha=0.05,max_iter=10000)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
print_metrics("Lasso Regression", y_test, y_pred_lasso, X_test)

# Ridge Regression
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
print_metrics("Ridge Regression", y_test, y_pred_ridge, X_test)

# ElasticNet Regression
elastic_net = ElasticNet(alpha=0.05, l1_ratio=0.5, max_iter=10000)
elastic_net.fit(X_train, y_train)
y_pred_elastic = elastic_net.predict(X_test)
print_metrics("ElasticNet Regression", y_test, y_pred_elastic, X_test)



Lasso Regression Results:
R² Score: 0.5742
Adjusted R²: 0.5731
MAE: 11.1730
MSE: 215.1085
RMSE: 14.6666

Ridge Regression Results:
R² Score: 0.5749
Adjusted R²: 0.5738
MAE: 11.1851
MSE: 214.7616
RMSE: 14.6547

ElasticNet Regression Results:
R² Score: 0.5503
Adjusted R²: 0.5492
MAE: 11.4315
MSE: 227.1792
RMSE: 15.0725


Underperformance

very low R² scores (around 0.005–0.006) and high RMSE values (~3.68)

regularization had minimal impact

crop_yield may be nonlinear or influenced by unobserved variables

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size=0.2, random_state=42)
print("=== X_train ===")
print(X_train.head())

print("\n=== X_test ===")
print(X_test.head())

print("\n=== y_train ===")
print(y_train.head())

print("\n=== y_test ===")
print(y_test.head())


# --- Train Random Forest model ---
rf_model = RandomForestRegressor(
    n_estimators=200,     # number of trees
    random_state=42,
    max_depth=None,       # you can tune later
    min_samples_split=2,
    min_samples_leaf=1
)
rf_model.fit(X_train, y_train)

# --- Predict ---
y_pred = rf_model.predict(X_test)

# --- Evaluate ---
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# --- Print results ---
print("Random Forest Regression Results:")
print(f"R² Score: {r2:.4f}")
print(f"Adjusted R²: {adj_r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")

# --- Show few predictions ---
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print("\nSample Predictions:")
print(results.head())


=== X_train ===
       crop_type  soil_type   soil_pH  temperature  humidity  wind_speed  \
13222  40.993453  31.322879 -1.346363     0.659199 -0.541632   -1.236411   
3295   39.424055  33.404045  0.182886     0.042697  0.358903    0.415329   
4401   39.434622  33.735036  1.712136    -1.237733  0.816279   -0.695849   
2310   37.025151  31.330653 -1.346363    -1.502800  0.816279    1.575467   
15429  37.107916  33.738559  1.712136    -1.884983  0.816279    0.784787   

              n         p         k  soil_quality  year  month  day  
13222  0.503248  0.112245 -0.002001     -0.541537  2019      3    7  
3295  -1.468169 -1.022104 -1.057300     -0.026147  2015      4    2  
4401  -0.092762  0.849571  0.877416     -1.294618  2015     10    9  
2310  -1.009700 -0.908669 -0.822789     -0.847476  2014     12    1  
15429  0.503248  0.225680 -0.177884     -1.332272  2020      1   24  

=== X_test ===
       crop_type  soil_type   soil_pH  temperature  humidity  wind_speed  \
20325  37.10334

In [13]:
import pandas as pd
raw_data=pd.read_csv("/kaggle/input/leaveoneout-encoded/leaveoneout_encoded")
raw_data

,Unnamed: 0,date,crop_type,soil_type,soil_pH,temperature,humidity,wind_speed,n,p,k,crop_yield,soil_quality
0,0,2014-01-01,40.966442,52.501041,6.50,20.052576,79.947424,8.591577,84.0,66.0,50.0,104.871310,66.666667
1,1,2014-01-01,37.007957,33.402460,6.75,19.751848,80.000000,2.682683,50.0,40.0,30.0,58.939796,35.000000
2,2,2014-01-01,36.923115,31.327259,5.50,16.110395,80.000000,7.696070,49.5,45.0,38.5,32.970413,22.166667
3,3,2014-01-01,41.984588,33.408309,6.75,14.826739,80.000000,10.366657,55.0,44.0,36.0,29.356115,39.375000
4,4,2014-01-01,32.445278,41.611338,6.25,18.323272,80.000000,8.198084,60.0,45.0,40.0,22.221375,42.291667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25490,25490,2023-12-31,37.026160,41.613125,6.25,11.617771,80.000000,14.700485,60.0,50.0,40.0,13.069169,43.750000
25491,25491,2023-12-31,41.967278,41.601361,6.25,19.538555,80.000000,3.666664,66.0,55.0,48.0,73.323885,49.291667
25492,25492,2023-12-31,40.999772,31.326036,5.50,21.068336,78.931664,8.795036,71.5,54.0,42.0,39.226521,27.916667
25493,25493,2023-12-31,37.421843,31.332520,5.50,11.079561,80.000000,10.969366,60.5,45.0,31.5,6.067881,22.833333


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# --- Use your existing processed data ---
rdata1 = raw_data.copy()
print(rdata1.head())

# --- Convert 'date' column to numeric (if exists) ---
if 'date' in rdata1.columns:
    rdata1['date'] = pd.to_datetime(rdata1['date'], errors='coerce')
    rdata1['year'] = rdata1['date'].dt.year
    rdata1['month'] = rdata1['date'].dt.month
    rdata1['day'] = rdata1['date'].dt.day
    rdata1 = rdata1.drop(columns=['date'])   
    
# Define features (X) and target (y)
x2 = rdata1.drop(columns=['crop_yield'])   # drop only the target column
y2 = rdata1['crop_yield']                  # target variable

print("Features")
print(x2.head())
print("  ")
print("Target")
print(y2.head())

   Unnamed: 0        date  crop_type  soil_type  soil_pH  temperature  \
0           0  2014-01-01  40.966442  52.501041     6.50    20.052576   
1           1  2014-01-01  37.007957  33.402460     6.75    19.751848   
2           2  2014-01-01  36.923115  31.327259     5.50    16.110395   
3           3  2014-01-01  41.984588  33.408309     6.75    14.826739   
4           4  2014-01-01  32.445278  41.611338     6.25    18.323272   

    humidity  wind_speed     n     p     k  crop_yield  soil_quality  
0  79.947424    8.591577  84.0  66.0  50.0  104.871310     66.666667  
1  80.000000    2.682683  50.0  40.0  30.0   58.939796     35.000000  
2  80.000000    7.696070  49.5  45.0  38.5   32.970413     22.166667  
3  80.000000   10.366657  55.0  44.0  36.0   29.356115     39.375000  
4  80.000000    8.198084  60.0  45.0  40.0   22.221375     42.291667  
Features
   Unnamed: 0  crop_type  soil_type  soil_pH  temperature   humidity  \
0           0  40.966442  52.501041     6.50    20.052

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=42)
print("=== X_train ===")
print(X_train.head())

print("\n=== X_test ===")
print(X_test.head())

print("\n=== y_train ===")
print(y_train.head())

print("\n=== y_test ===")
print(y_test.head())


# --- Train Random Forest model ---
rf_model = RandomForestRegressor(
    n_estimators=200,     # number of trees
    random_state=42,
    max_depth=None,       # you can tune later
    min_samples_split=2,
    min_samples_leaf=1
)
rf_model.fit(X_train, y_train)

# --- Predict ---
y_pred = rf_model.predict(X_test)

# --- Evaluate ---
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# --- Print results ---
print("Random Forest Regression Results:")
print(f"R² Score: {r2:.4f}")
print(f"Adjusted R²: {adj_r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")

# --- Show few predictions ---
results = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print("\nSample Predictions:")
print(results.head())


=== X_train ===
       Unnamed: 0  crop_type  soil_type  soil_pH  temperature   humidity  \
13222       13222  40.993453  31.322879     5.50    25.112135  74.887865   
3295         3295  39.424055  33.404045     6.75    21.721884  78.278116   
4401         4401  39.434622  33.735036     8.00    14.680577  80.000000   
2310         2310  37.025151  31.330653     5.50    13.222928  80.000000   
15429       15429  37.107916  33.738559     8.00    11.121236  80.000000   

       wind_speed     n     p     k  soil_quality  year  month  day  
13222    6.289921  71.5  54.0  42.0     27.916667  2019      3    7  
3295    11.249571  50.0  44.0  33.0     37.041667  2015      4    2  
4401     7.913056  65.0  60.5  49.5     14.583333  2015     10    9  
2310    14.733094  55.0  45.0  35.0     22.500000  2014     12    1  
15429   12.358934  71.5  55.0  40.5     13.916667  2020      1   24  

=== X_test ===
       Unnamed: 0  crop_type  soil_type  soil_pH  temperature   humidity  \
20325       203

The Leave-One-Out Encoded version (without MinMax scaling) is marginally better and simpler, so you can confidently use that as your final model for reporting.

Leave-One-Out Encoding (without MinMax) has a slightly lower MSE and RMSE, meaning:

The model’s predictions are marginally more accurate.

It handled categorical data encoding perfectly on its own.